In [ ]:
%matplotlib inline


NLP From Scratch: Classifying Names with a Character-Level RNN
**************************************************************
**Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>`_

我们会建立和训练基础的字符水平RNN来给词语分类。这份教程以及接下来的两份教程会展示怎么从零开始建立nlp模型的数据预处理，特别是不会使用很多方便的`torchtext`函数，这样你可以看到你nlp模型是怎么在低水平任务上工作的。

一个字符水平的RNN通过对每一步的字符输出作为预测的序列读取文字。我们取最后的预测作为输出，比如说这个词属于哪一类。

特别地，我们会从18种语言中训练许许多多的姓氏，以及预测哪种语言的名字是从拼写基础开始的:
::

    $ python predict.py Hinton
    (-0.47) Scottish
    (-1.52) English
    (-3.57) Irish

    $ python predict.py Schmidhuber
    (-0.19) German
    (-2.48) Czech
    (-2.68) Dutch


**Recommended Reading:**

我假定你有安装PyTorch，知道Python，以及懂得Tensors:

-  https://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user

这样会同时对熟悉RNN的工作原理有用:

-  `这种RNN网络不合理的有效性<https://karpathy.github.io/2015/05/21/rnn-effectiveness/>`__
   展示一大堆真实生活中的例子
-  `理解LSTM神经网络 <https://colah.github.io/posts/2015-08-Understanding-LSTMs/>`__
   是特别关于LSTMs同时是普通RNNs的有效性
   

数据预处理
==================

.. 注意::
   从这里下载数据
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   以及把它抽取到现在的目录下.

包括了 ``data/names`` 目录是18个文本文件，命名为
"[Language].txt".每一个文件包括了一大堆名字，一个名字代表一行，大多罗马化了(但是我们仍然需要从Unicode转化到ASCII)。

我们会在每一种语言都有一个列表的名字来结尾``{language: [names ...]}``。这种形式的变量"category"和"line"(我们这里是对于语言和名字而言)可以作为后期的扩展。


In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

[]
Slusarski


现在我们有category_lines,一个字典用来定位每一个类别(语言)到一行的一个列表。我们同时跟踪all_categories(只是一个语言的列表)和n_categories对于接下来的引用。

In [ ]:
print(category_lines['Italian'][:5])

把名字转换为张量
--------------------------

现在我们有所有的名字机构，我们需要把它们转化进张量以便后续使用。

为了表示单独的字母，我们使用独热向量的尺寸``<1 x n_letters>``。一个独热向量除了一个1，其他都是0，对于目前字母的序列来说，比如``"b" = <0 1 0 0 0 ...>``。

为了让创造一个词语我们把一大堆这些转化成2D矩阵``<line_length x 1 x n_letters>``。

那些额外1维是因为PyTorch假定所有东西是在一个batches的，我们只是使用一个1个batch。


In [3]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


Creating the Network
====================
在自动微分之前，在Torch里创建一个rnn在几步timesteps中启动克隆每一层的参数。这层有隐藏状态以及梯度，这些现在完全被图控制。这意味着你可以在一个非常纯净的方式里利用一个rnn作为一个日常feed-forward层。

这个RNN模块(更多地从`the PyTorch for Torch users
tutorial <https://pytorch.org/tutorials/beginner/former_torchies/
nn_tutorial.html#example-2-recurrent-net>`__里复制过来)
只是2层线性层，在输入和隐藏状态里操作，在输出后带有LogSoftmax。

.. figure:: https://i.imgur.com/Z2xbySO.png
   :alt:

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

为了运行这个神经网络我们需要去通过一个输入(在这个例子中，是现在字母的张量)和一个之前的隐藏状态(这个我们初始化作为零)。我们把输出(每种语言的概率)和下一个隐藏状态(我们在下一步时候保存了)

In [ ]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

为了效率我们不希望每一步都创建一个新的张量，所以我们会使用``lineToTensor``而不是``letterToTensor``以及使用切片。这样可以进一步优化之前每一个batch里的张量。

In [ ]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

正当你可以看到输出是一个 ``<1 x n_categories>`` 张量时, 每一个item都是这个类别下的最大似然(最高概率就是最大可能)。

Training
========
Preparing for Training
----------------------
在进行训练之前我们应该做一些helper函数。这首先是去翻译网络的输出，这里我们知道是每一个类别的最大似然，我们可以使用``Tensor.topk``来得到最大值的index:


In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

我们会同时希望一个快速的方法来取得一个训练案例(一个名字和它的语言):

In [ ]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

Training the Network
--------------------
现在所有训练模型所需要的就是给它展示一大堆例子，让它去推测，以及在它错误的时候去告诉它。

因为损失函数``nn.NLLLoss`` 是不恰当的，因为最后一层的RNN是``nn.LogSoftmax``。


In [ ]:
criterion = nn.NLLLoss()

每一个训练的循环会:

-  创建一个输入和目标张量
-  创建一个初始化为零的隐藏层
-  阅读每一个字母在

   -  保存在隐藏状态中的下一个字母

-  比较最终的输出和目标
-  反向传播
-  返回输出和损失




In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

现在我们去运行那一大堆例子。因为``train``函数同时返回了输出和损失，我们可以打印它的输出以及同时保持损失图。因为这里有1000多个案例我们可以打印仅仅每一个``print_every``例子，然后得到平均损失。

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

Plotting the Results
--------------------
从``all_losses``打印之前所有损失，展示神经网络学习过程:


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

Evaluating the Results
======================

为了看到神经网络的在不同类别上的表现，我们会创建一个混淆矩阵展示神经网络的预测(列)以及每一种语言(行)。为了计算混淆矩阵，一大堆例子运行了
``evaluate()``,这跟``train()``一样，反向传播在不断减少。

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

你可以选择在坐标轴上的亮点来表示预测错误的语言。比如说中文对韩文，西班牙语对意大利语。它看上去对于希腊语来说是最准的，对于英语来说是最差的(可能因为跟其他语言有重合)。

Running on User Input
---------------------




In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')

这个脚本最后的版本`在使用的PyTorch
repo中 <https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification>`__
把上面的代码写进几个文档中:

-  ``data.py`` (loads files)
-  ``model.py`` (defines the RNN)
-  ``train.py`` (runs training)
-  ``predict.py`` (runs ``predict()`` with command line arguments)
-  ``server.py`` (serve prediction as a JSON API with bottle.py)

运行 ``train.py`` 来训练和保存网络。

运行 ``predict.py`` 带着名字来看所有的预测结果:

::

    $ python predict.py Hazaki
    (-0.42) Japanese
    (-1.39) Polish
    (-3.51) Czech

运行``server.py`` 以及访问 http://localhost:5533/Yourname 来得到 JSON
预测值的输出。

Exercises
=========

-  尝试line -> category的不同数据集, 比如说:

   -  任何词语-> 语言
   -  名字 -> 性别
   -  角色名字 -> 作家
   -  页面标题 -> 博客或者讨论区

-  得到结果更好或者更大形状的网络

   -  加入更多的线性层
   -  尝试``nn.LSTM``以及``nn.GRU``层
   -  结合这些RNNs模型得到的不同结果作为更高级的网络
